In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import numpy as np
import re
import random
import matplotlib.pyplot as plt
import ast
import json
from collections import Counter


client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
    )
pd.set_option('display.max_colwidth', None)

In [ ]:
prompt="During a lunch break, you are in the college cafeteria where various meal options are available. Please choose your lunch.\n\
You don't have to choose from the entire menu.Please think about how much you eat and take the right amount.\n\
When responding, please consider each option independently to ensure your selection is based solely on your preference.\n\
Answer only the menu you chose.\n\
"

In [3]:
def generate_completion(role, choice, prompt):
    response = client.chat.completions.create(
        model="gpt-4-turbo-2024-04-09",
        #model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": f"{prompt},{choice}"}
        ],
        temperature=0.2,
        max_tokens=100,
        )
    return response

# 各単語の出現回数をカウント
def count_words(text, words_list):
    counts = {word: text.count(word) for word in words_list}
    food_selected = []

    # 結果を表示
    for food, count in counts.items():
        if count > 0:
            food_selected.append(food)
            break
    return food_selected

def buffet_vegetable_preference(role,prompt,range_number):
    global food_list

    df = pd.DataFrame()

    for i in range(range_number):

        buffet_prompt = buffet.generate_buffet_prompt()

        response = generate_completion(role, buffet_prompt, prompt)
        content = response.choices[0].message.content

        #extract number from responce
        counts = {food: content.count(food) for food in food_list}


        df_sample = pd.DataFrame({"attribute":role,"response":content,"number":[counts]},index=[0])

        df = pd.concat([df,df_sample],ignore_index=True)
    return df

def count_number(df):
    result_dict = Counter()
    for i in range(len(df)):
        dic_1 = ast.literal_eval(df["number"][i])
        # Update the result counter by adding counts from the current dictionary
        result_dict.update(dic_1)
    return result_dict

def plot_bar(result_dict):
    # Creating a bar plot
    plt.figure(figsize=(10, 5))  # Set the figure size (optional)
    plt.bar(result_dict.keys(), result_dict.values(), color='blue')  # Create a bar plot

    plt.xlabel('Keys')  # Label on X-axis
    plt.ylabel('Counts')  # Label on Y-axis
    plt.title('Bar plot of Result Dictionary')  # Title of the plot

    plt.xticks(rotation=270)  # Rotate x-axis labels for better visibility if required
    plt.grid(True)  # Add a grid for better readability
    plt.show()  # Display the plot

def calculate_confidence_interval(dict_1, dict_2, dict_3):
    # すべてのキーを取得します
    keys = list(dict_1.keys())

    # すべての辞書の値を配列にまとめます
    values = np.array([[dict_1[key], dict_2[key], dict_3[key]] for key in keys])

    # 平均値を計算します
    mean_values = np.mean(values, axis=1)
    std_values = np.std(values, axis=1)

    # 95%信頼区間を計算します
    confidence_interval = 1.96 * np.std(values, axis=1) / np.sqrt(values.shape[1])

    lower_bounds = mean_values - confidence_interval
    upper_bounds = mean_values + confidence_interval

    results = pd.DataFrame({
        'Mean': mean_values,
        'Standard': std_values,
        'Lower Bound (95% CI)': lower_bounds,
        'Upper Bound (95% CI)': upper_bounds
    },  columns=keys)

    return results,mean_values,confidence_interval
# プロット
def plot_bar_with_CI(mean_values,confidence_interval):
    global food_list

    x_pos = np.arange(len(food_list))

    plt.figure(figsize=(12, 8))
    plt.bar(x_pos, mean_values, yerr=confidence_interval, align='center', alpha=0.7, capsize=10)
    plt.xticks(x_pos, food_list, rotation='vertical')
    plt.xlabel('Menu Items')
    plt.ylabel('Scores')
    plt.title('Menu Items with 95% Confidence Interval')
    plt.show()


In [2]:
import random

class Buffet_UTokyo:
    def __init__(self):
        self.choice_1 = "Dynamite chili and tangy lime-seasoned beets"
        self.choice_2 = "Beets"
        self.choice_3 = "Lighter-choice beets with no added sugar"
        self.choice_4 = "High-antioxidant beets"
        self.choice_list = [self.choice_1, self.choice_2, self.choice_3, self.choice_4]

        self.class_1 = "Main dish"
        self.class_2 = "Side dish, Salad"
        self.class_3 = "Rice"
        self.class_4 = "Noodles"

        # Main dish
        self.food_1 = [
            "Deep-fried succulent chicken",#手仕込みジューシー唐揚げ
            "Hamburger steak with arrabbiata sauce",#スパイシートマトハンバーグ
        #    "Grilled chicken with salt-based sauce",#グリルチキン（塩だれ）
            "Salt-grilled mackerel"#さば塩焼き
        ]
        # Side dish, Salad
        self.food_2 = [
            "French fries Spicy",#フライドポテト（スパイシー）
        #    "Soft-boiled egg",#温泉たまご
            "Boiled spinach with dried bonito",#ほうれん草のおひたし
        #    "Boiled chopped okura",#オクラのおひたし
        #    "Soy sauce taste boiled dried daikon radish strips",#切り干し大根
        #    "Tuna salad (with boiled egg, wakame seaweed, and shredded cabbage)",#ツナサラダ
            "Macaroni and corn salad (with wakame seaweed and shredded cabbage)"#マカロニ＆コーンサラダ
        ]
        # Rice
        self.food_3 = [
            "(Small size) Rice",#ごはん(小)
            "(Medium size) Rice",#ごはん(中)
            "(Large size) Rice"#ごはん(大)
        ]
        # Noodles
        self.food_4 = [
            "Plain udon noodles",#かけうどん
            "Zaru soba cool soba noodles",#ざるそば
            "Ramen in soy-sauce flavored soup"#醤油ラーメン
        ]

    def generate_buffet_prompt(self):
        #random_choice = "Dynamite chili and tangy lime-seasoned beets"#random.choice(self.choice_list)
        #self.food_2.append(random_choice)

        # Shuffle each food category list to randomize their order
        random.shuffle(self.food_1)
        random.shuffle(self.food_2)
        random.shuffle(self.food_3)
        random.shuffle(self.food_4)

        
        buffet_prompt = f"{self.class_1}\n" + "\n".join(f"{idx+1}. {dish}" for idx, dish in enumerate(self.food_1)) + "\n" \
                         f"{self.class_2}\n" + "\n".join(f"{idx+1}. {dish}" for idx, dish in enumerate(self.food_2)) + "\n" \
                         f"{self.class_3}\n" + "\n".join(f"{idx+1}. {dish}" for idx, dish in enumerate(self.food_3)) + "\n" \
                         f"{self.class_4}\n" + "\n".join(f"{idx+1}. {dish}" for idx, dish in enumerate(self.food_4))
        
        return buffet_prompt

# Instance creation and function call
buffet = Buffet_UTokyo()

food_list = buffet.food_1 + buffet.food_2 + buffet.food_3 + buffet.food_4
print(food_list)
buffet_prompt = buffet.generate_buffet_prompt()
print(buffet_prompt)




['Deep-fried succulent chicken', 'Hamburger steak with arrabbiata sauce', 'Salt-grilled mackerel', 'French fries Spicy', 'Boiled spinach with dried bonito', 'Macaroni and corn salad (with wakame seaweed and shredded cabbage)', '(Small size) Rice', '(Medium size) Rice', '(Large size) Rice', 'Plain udon noodles', 'Zaru soba cool soba noodles', 'Ramen in soy-sauce flavored soup']
Main dish
1. Deep-fried succulent chicken
2. Hamburger steak with arrabbiata sauce
3. Salt-grilled mackerel
Side dish, Salad
1. Macaroni and corn salad (with wakame seaweed and shredded cabbage)
2. Boiled spinach with dried bonito
3. French fries Spicy
Rice
1. (Medium size) Rice
2. (Large size) Rice
3. (Small size) Rice
Noodles
1. Plain udon noodles
2. Zaru soba cool soba noodles
3. Ramen in soy-sauce flavored soup


In [ ]:
dict_1 = count_number(pd.read_csv("df_baseline_1.csv"))
dict_2 = count_number(pd.read_csv("df_baseline_2.csv"))
dict_3 = count_number(pd.read_csv("df_baseline_3.csv"))